In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristics Random Forest



In [2]:
#decide which df to use
df = pd.read_csv("Dataframes/characteristics.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...
19332,AVY.N,2014-01-01,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19333,AVY.N,2013-10-01,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19334,AVY.N,2013-07-01,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19335,AVY.N,2013-04-01,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


#### OLS Regression

In [3]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.value_counts("loss firm status")


loss firm status
-1    17281
 1     1271
 0       14
dtype: int64

In [4]:
df_accuracy_new

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...
19332,AVY.N,2014-01-01,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19333,AVY.N,2013-10-01,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19334,AVY.N,2013-07-01,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19335,AVY.N,2013-04-01,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


In [5]:
df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                                 4.196      1.615339e+09   
           2022-07-01                                 1.503      2.055818e+09   
           2022-04-01                                34.342      1.412650e+09   
           2022-01-01                                40.267      1.035557e+09   
           2021-10-01                                17.194      1.411448e+09   
...                                                     ...               ...   
AVY.N      2014-01-01                                 1.471      1.583900e+09   
           2013-10-01                                 8.095      1.504900e+09   
           2013-07-01                                 1.068      1.552300e+09   
           2013-04-01                                 2.482      1.498900e+09   
           2013-01-01                                11.178      1.532200e+09   

                       Enterprise Value  Market Capitalization  \
Instrument Date                                                  
POOL.OQ    2022-10-01      1.406160e+10           1.259813e+10   
           2022-07-01      1.556187e+10           1.405795e+10   
           2022-04-01      1.843682e+10           1.696712e+10   
           2022-01-01      2.384882e+10           2.268979e+10   
           2021-10-01      1.769954e+10           1.742020e+10   
...                                 ...                    ...   
AVY.N      2014-01-01      5.553068e+09           4.877168e+09   
           2013-10-01      5.029514e+09           4.273414e+09   
           2013-07-01      5.436229e+09           4.258229e+09   
           2013-04-01      5.459128e+09           4.309428e+09   
           2013-01-01      4.413001e+09           3.426001e+09   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
POOL.OQ    2022-10-01                                           2.383926     
           2022-07-01                                           2.749481     
           2022-04-01                                           3.422880     
           2022-01-01                                           4.504382     
           2021-10-01                                           3.684400     
...                                                                  ...     
AVY.N      2014-01-01                                           0.920659     
           2013-10-01                                           0.903176     
           2013-07-01                                           0.906078     
           2013-04-01                                           0.895787     
           2013-01-01                                           0.799347     

                       3 Month Total Return    Volume  loss firm status  
Instrument Date                                                          
POOL.OQ    2022-10-01             -9.164375  238539.0                -1  
           2022-07-01            -21.661570  185584.0                -1  
           2022-04-01            -25.160664  290897.0                -1  
           2022-01-01             30.489705   82349.0                -1  
           2021-10-01             -5.130322  157740.0                -1  
...                                     ...       ...               ...  
AVY.N      2014-01-01             16.003852  157761.0                -1  
           2013-10-01              2.467620  236291.0                -1  
           2013-07-01             -0.054218       0.0                -1  
           2013-04-01             24.148726       0.0                -1  
           2013-01-01             10.637479  207295.0                -1  

[18566 rows x 8 columns]

In [7]:
# Import libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Load data

X_var_names = ["Revenue - Actual", "Enterprise Value", "Market Capitalization", "loss firm status",
               "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]

# Define features and target
X = df_accuracy_new[X_var_names]
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Create and fit random forest classifier
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

# Make predictions on test set
y_pred = rf.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}')

ValueError: continuous is not supported